In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from datetime import datetime as dt
from dotenv import load_dotenv
import os
from multielo import MultiElo

In [13]:
load_dotenv()

sheet_key = os.getenv('key')

df_url = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_key}/export?format=csv')

df_matches = pd.DataFrame(df_url).set_index('Name')
df_p_res = pd.read_csv('Match_Res.csv').set_index('Name')
df_p_res = pd.DataFrame(df_p_res)
#df_matches['Elo'] = df_matches['Elo'].fillna(df_p_res)


#m = df_p_res.merge(df_matches, on='Name', how='inner', suffixes=['', '_'], indicator=True)


#df_matches.to_csv('Match_data.csv')

df_matches = df_matches[df_matches['Elo'].isnull()]
df_matches.to_csv('Match_data.csv')

In [14]:
df_matches

,Date,Result,Match,Elo,New_Elo
Name,,,,,
Ricardo,08-26-2024,2,1,NaN,NaN
Caleb,08-26-2024,1,1,NaN,NaN
Sebas,08-26-2024,3,1,NaN,NaN
Ricardo,08-26-2024,3,2,NaN,NaN
Caleb,08-26-2024,2,2,NaN,NaN
Sebas,08-26-2024,1,2,NaN,NaN


## Si vas a comenzar desde 0 pasa estas lineas a codigo




headerL = ['Name','Elo','Wins']
Leaderboard_n = [['Sebas',1200,np.nan],['Ricardo',1200,np.nan],['Jochy',1200,np.nan],['Jael',1200,np.nan],['Caleb',1200,np.nan]]
LB_dfp = pd.DataFrame(Leaderboard_n)
LB_dfp.columns = headerL
LB_dfp = LB_dfp.set_index('Name')
LB_dfp
LB_dfp.to_csv('Leaderboard.csv')

In [15]:
df = pd.read_csv('Match_data.csv')
df['Date'] = pd.to_datetime(df['Date'])

loaded_LB_df = pd.read_csv('Leaderboard.csv')

#loaded_LB_df = loaded_LB_df.set_index('Name')


In [16]:
df1 = df[df.Date == df.Date.max()]

df_collec = {y: df1[df1['Match']==y] for y in df1['Match'].unique()}



In [17]:
df_collec

{1:       Name       Date  Result  Match  Elo  New_Elo
 0  Ricardo 2024-08-26       2      1  NaN      NaN
 1    Caleb 2024-08-26       1      1  NaN      NaN
 2    Sebas 2024-08-26       3      1  NaN      NaN,
 2:       Name       Date  Result  Match  Elo  New_Elo
 3  Ricardo 2024-08-26       3      2  NaN      NaN
 4    Caleb 2024-08-26       2      2  NaN      NaN
 5    Sebas 2024-08-26       1      2  NaN      NaN}

In [18]:

match_res = []

for i in range(1,len(df_collec)+1):

    loaded_LB_df = pd.read_csv('Leaderboard.csv')
    df_i = pd.DataFrame(df_collec[i].set_index('Name'))
    lb = loaded_LB_df.set_index('Name')['Elo']

    
    df_i['Elo'] = df_i['Elo'].fillna(lb)
    df_i = df_i.sort_values(by=['Result'])

    lb = pd.DataFrame(lb)
    c_elo = df_i.set_index('Result')['Elo']
    c_elo = np.array(c_elo)

    elo = MultiElo()

    n_elo = elo.get_new_ratings(c_elo)

    df_i['New_Elo'] = n_elo

    res_elo = df_i.drop(['Date','Result','Match'],axis=1)
    

    lb = pd.merge(lb, res_elo, on = ['Name', 'Elo'], how = 'outer')

    lb['Elo'] = lb['New_Elo'].fillna(lb['Elo'])
    lb['Wins'] = np.nan
    lb = lb.drop('New_Elo',axis=1)

    match_res.append(df_i)

    lb['Elo'] = lb['Elo'].astype(int)

    lb.to_csv('Leaderboard.csv')

    #print(res_elo)
     
match_res = pd.concat(match_res)

load_match_res = pd.read_csv('Match_Res.csv')

lmr = load_match_res.set_index('Name')

print(lmr)

lmr['Date'] = pd.to_datetime(lmr['Date'])

match_res_p = lmr.append(match_res)

match_res_p.to_csv('Match_Res.csv')

##print(lmr) 
#print(match_res_p)
print(match_res_p)


               Date  Result  Match     Elo      New_Elo
Name                                                   
Sebas    2024-07-23       1      1  1200.0  1224.000000
Ricardo  2024-07-23       2      1  1200.0  1208.000000
Jochy    2024-07-23       3      1  1200.0  1192.000000
Jael     2024-07-23       4      1  1200.0  1176.000000
Ricardo  2024-07-23       1      2  1208.0  1231.449228
Sebas    2024-07-23       2      2  1224.0  1229.981841
Jael     2024-07-23       3      2  1176.0  1170.384756
Caleb    2024-07-23       4      2  1200.0  1176.184174
              Date  Result  Match     Elo      New_Elo
Name                                                  
Sebas   2024-07-23       1      1  1200.0  1224.000000
Ricardo 2024-07-23       2      1  1200.0  1208.000000
Jochy   2024-07-23       3      1  1200.0  1192.000000
Jael    2024-07-23       4      1  1200.0  1176.000000
Ricardo 2024-07-23       1      2  1208.0  1231.449228
Sebas   2024-07-23       2      2  1224.0  1229.981841


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21300\1483683025.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  match_res_p = lmr.append(match_res)


In [19]:
print(match_res_p)

              Date  Result  Match     Elo      New_Elo
Name                                                  
Sebas   2024-07-23       1      1  1200.0  1224.000000
Ricardo 2024-07-23       2      1  1200.0  1208.000000
Jochy   2024-07-23       3      1  1200.0  1192.000000
Jael    2024-07-23       4      1  1200.0  1176.000000
Ricardo 2024-07-23       1      2  1208.0  1231.449228
Sebas   2024-07-23       2      2  1224.0  1229.981841
Jael    2024-07-23       3      2  1176.0  1170.384756
Caleb   2024-07-23       4      2  1200.0  1176.184174
Caleb   2024-08-26       1      1  1176.0  1200.622586
Ricardo 2024-08-26       2      1  1231.0  1229.264001
Sebas   2024-08-26       3      1  1229.0  1206.113413
Sebas   2024-08-26       1      2  1206.0  1227.854241
Caleb   2024-08-26       2      2  1200.0  1201.072459
Ricardo 2024-08-26       3      2  1229.0  1206.073299


In [20]:
Leaderboard = pd.read_csv('Leaderboard.csv')
Matches = pd.read_csv('Match_Res.csv')

Leaderboard = Leaderboard.set_index('Name')

Leaderboard = Leaderboard.sort_values(by=['Elo'],ascending=False)


m = Matches.drop('Date', axis=1)
#m = m[m.Result < 2]

m = m[m['Result'] == 1]['Name'].value_counts()

Leaderboard['Wins'] = m
Leaderboard['Wins'] = Leaderboard['Wins'].fillna(0)

Leaderboard
#print(m)

,Elo,Wins
Name,,
Sebas,1227,2.0
Ricardo,1206,1.0
Caleb,1201,1.0
Jochy,1192,0.0
Jael,1170,0.0


,Name,Date,Result,Match,Elo,New_Elo
0,Sebas,2024-07-23,1,1,1200.0,1224.000000
1,Ricardo,2024-07-23,2,1,1200.0,1208.000000
2,Jochy,2024-07-23,3,1,1200.0,1192.000000
3,Jael,2024-07-23,4,1,1200.0,1176.000000
4,Ricardo,2024-07-23,1,2,1208.0,1231.449228
5,Sebas,2024-07-23,2,2,1224.0,1229.981841
6,Jael,2024-07-23,3,2,1176.0,1170.384756
7,Caleb,2024-07-23,4,2,1200.0,1176.184174
8,Caleb,2024-08-26,1,1,1176.0,1200.622586
9,Ricardo,2024-08-26,2,1,1231.0,1229.264001
